# 1. train_test : training + dev set


1. 선다형 : 선지 균등
2. 단답형 : # 기준으로 나누고 띄어쓰기를 없앴을 때 다른 문자열이 존재한다면 최대 3개의 답변 학습 -> 가장 긴 것으로 대체
3. 서술형 : 그대로 학습

# 학습 프롬프트

1. 선다형 + 단답형 : detailed
2. 서술형 : ...


# 1차 SFT 후 train set에서 어려운 문제 RL 학습
- train set을 8번 추론 -> 정답률 100% 미만인 데이터 추출
    - 서술형은 제외하고 선다형과 단답형으로만 추출
    - 서술형은 전부 포함한다.

- RL 시 보상함수
    - exact match:
        선다형 + 단답형
    - 강건한(robust) 평가지표가 있으면 좋을 것 같은데 bleu? rouge?


In [5]:
import pandas as pd

In [6]:
training_df = pd.read_json("data/train.json", encoding="utf-8-sig")
dev_df = pd.read_json("data/dev.json", encoding="utf-8-sig")
test_df = pd.read_json("data/test.json", encoding="utf-8-sig")

len(training_df), len(dev_df), len(test_df)

(491, 98, 394)

In [7]:
# training_df = pd.concat([training_df, dev_df], ignore_index=True)
# training_df

In [8]:
training_df = pd.concat([pd.DataFrame(training_df['input'].to_list()), pd.DataFrame(training_df['output'].to_list())], axis=1)
dev_df = pd.concat([pd.DataFrame(dev_df['input'].to_list()), pd.DataFrame(dev_df['output'].to_list())], axis=1)
# test_df = pd.concat([pd.DataFrame(test_df['input'].to_list()), pd.DataFrame(test_df['output'].to_list())], axis=1)

In [ ]:
# training_df = pd.concat([pd.DataFrame(training_df['input'].to_list()), pd.DataFrame(training_df['output'].to_list())], axis=1)
# training_df

,category,domain,question_type,topic_keyword,question,answer
0,문화 지식,예술,선다형,전통놀이,다음 한국의 전통 놀이 중 '조선시대'에 행한 놀이는? \n 1\t주사위 놀이 2\...,3
1,문화 지식,풍습/문화유산,선다형,진도아리랑,한국의 진도아리랑은 어떤 타령에 영향을 받아 만들어졌나요? \n 1\t 각설이타령 ...,4
2,문화 지식,사회,서술형,행정구역,대한민국의 행정구역 체계를 서술하세요.,"대한민국의 행정구역은 여러 종류의 지역 단위로 나뉘어 구성되어 있으며, 먼저 특별시..."
3,문화 지식,과학기술,선다형,우주인,힌국인 최초의 우주 비행사는 누구인가요? \n 1\t 조니 김 2\t 이소연...,2
4,문화 지식,일상생활,선다형,돌잔치,돌잔치와 관련 없는 것을 고르세요. \n 1\t 돌잡이 2\t 돌상 3\t 돌...,4
...,...,...,...,...,...,...
584,문화 관점,가치관,단답형,이사 미신,러시아에서는 이사한 집에 고양이가 드나들면 복이 온다고 합니다. 한국에서는 복을 위...,밥솥
585,문화 지식,역사,선다형,병인양요,흥선군의 천주교 탄압에 대한 보복으로 프랑스군이 강화도에 침입한 사건은? \n 1\...,2
586,문화 지식,역사,선다형,고려시대사,다음 중 고려시대의 사건이 아닌 것은 무엇인가요? \n 1\t이자겸의 난 2\t저고...,4
587,문화 실행,지리,단답형,기후동행카드,기후동행카드를 실물로 사용하기 위한 방법을 순서에 맞게 배열하세요. 배열 시 각 기...,ㄷ-ㄴ-ㄹ-ㄱ#ㄷ - ㄴ - ㄹ - ㄱ


### 1. \t -> '.'

In [9]:
import re

def convert_number_tab_to_dot(text):
    # 정규식: 숫자 + 탭을 숫자 + 점으로 변경
    return re.sub(r'(\d+)\\t', r'\1.', text)


training_df['question'] = training_df['question'].apply(convert_number_tab_to_dot)
training_df['question'] = training_df['question'].str.replace('\\n', '\n')

dev_df['question'] = dev_df['question'].apply(convert_number_tab_to_dot)
dev_df['question'] = dev_df['question'].str.replace('\\n', '\n')

training_df

,category,domain,question_type,topic_keyword,question,answer
0,문화 지식,예술,선다형,전통놀이,다음 한국의 전통 놀이 중 '조선시대'에 행한 놀이는? \n 1.주사위 놀이 2.검...,3
1,문화 지식,풍습/문화유산,선다형,진도아리랑,한국의 진도아리랑은 어떤 타령에 영향을 받아 만들어졌나요? \n 1. 각설이타령 ...,4
2,문화 지식,사회,서술형,행정구역,대한민국의 행정구역 체계를 서술하세요.,"대한민국의 행정구역은 여러 종류의 지역 단위로 나뉘어 구성되어 있으며, 먼저 특별시..."
3,문화 지식,과학기술,선다형,우주인,힌국인 최초의 우주 비행사는 누구인가요? \n 1. 조니 김 2. 이소연 ...,2
4,문화 지식,일상생활,선다형,돌잔치,돌잔치와 관련 없는 것을 고르세요. \n 1. 돌잡이 2. 돌상 3. 돌옷 ...,4
...,...,...,...,...,...,...
486,문화 지식,풍습/문화유산,선다형,비파,"중앙아시아 지역에서 기원한 전통 현악기로 중국, 일본, 한국 등 동양에서 주로 연주...",3
487,문화 관점,예술,서술형,서예,한국의 ‘서예’와 이슬람 문화권의 ‘캘리그래피’가 공유하는 조형상의 특징을 서술하고...,한국의 서예와 이슬람 문화권의 캘리그래피는 모두 문자를 단순한 전달 수단이 아닌 예...
488,문화 지식,과학기술,선다형,5G,한국 정부에서 추진하는 5G 기반 5대 핵심서비스가 아닌 것은? \n 1.자율주행차...,2
489,문화 지식,정치/경제,선다형,예금자보호제도,예금자보호제도에 의해 금융회사의 파산 시 한국에서 예금을 보호받을 수 있는데 그 한...,4


In [10]:
training_df['answer'].value_counts()

answer
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                   73
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                   67
1                                                                                  

In [11]:
training_df[training_df['question_type'] == '단답형']

,category,domain,question_type,topic_keyword,question,answer
5,문화 지식,풍습/문화유산,단답형,금줄,한국의 금줄은 무엇을 막기 위해 문이나 길 어귀에 건너질러 매나요?,부정
12,문화 실행,일상생활,단답형,상차림,한국 상차림 시 식기류를 어떻게 배열하는지 왼쪽을 기준으로 순서대로 나열하세요. 배...,ㄴ-ㄷ-ㄹ-ㄱ#ㄴ - ㄷ - ㄹ - ㄱ
13,문화 지식,역사,단답형,조선후기 실학자(정약용),조선 후기의 실학 사상가로 목민심서를 쓴 인물은?,정약용
14,문화 지식,과학기술,단답형,이태규,한국에 양자역학을 화학 반응 연구에 도입한 과학자로 이론 화학을 연구했던 과학자는 ...,이태규
16,문화 지식,역사,단답형,고조선,한국사에서 처음으로 등장한 국가는 무엇인가요?,고조선
...,...,...,...,...,...,...
473,문화 지식,예술,단답형,아니리,판소리에서 창자가 소리를 하다가 한 대목에서 다른 대목으로 넘어가기 전에 자유리듬으...,아니리
477,문화 지식,지리,단답형,신도시,충청남도 홍성과 예산에 개발된 신도시로 지역의 균형적 발전을 이루기 위해 충남의 행...,내포 신도시#내포신도시#내포
478,문화 지식,일상생활,단답형,비유 표현,한국에서는 교활하고 약삭빠른 사람을 어떤 동물에 비유하나요?,여우
481,문화 지식,예술,단답형,서울아트시네마,"2005년에 개관하였으며, 교육 및 문화적 목적으로 영화를 상영하는 서울의 유일한 ...",서울아트시네마#서울 아트 시네마#서울아트 시네마#서울 아트시네마


In [8]:
tmp_df = training_df[training_df['question_type'] == '단답형']
tmp_df[tmp_df['answer'].str.contains('#')]

,category,domain,question_type,topic_keyword,question,answer
12,문화 실행,일상생활,단답형,상차림,한국 상차림 시 식기류를 어떻게 배열하는지 왼쪽을 기준으로 순서대로 나열하세요. 배...,ㄴ-ㄷ-ㄹ-ㄱ#ㄴ - ㄷ - ㄹ - ㄱ
25,문화 지식,교육,단답형,초등학교,한국의 보편적인 취학 시작 연령은? 만 나이로 이야기하세요.,만 6세#만6세#만6살#만 6살#6세#6살#여섯살
34,문화 지식,예술,단답형,타령,"한국의 타령 중 판소리의 장단은 주로 2가지인데, 하나는 중중모리 장단이고 다른 하...",자진모리장단#자진모리 장단
42,문화 실행,풍습/문화유산,단답형,식혜,식혜 만드는 방법을 순서에 맞게 배열하세요. 배열 시 각 기호는 '-'로 연결합니다...,ㅁ-ㄱ-ㄷ-ㄴ-ㄹ#ㅁ - ㄱ - ㄷ - ㄴ - ㄹ
44,문화 실행,지리,단답형,지하철 노선,서울 지하철 신림선을 서원역에서 샛강 방향으로 탔을 때 지나가는 역을 순서대로 나열...,ㄱ-ㄹ-ㄴ-ㄷ#ㄱ - ㄹ - ㄴ - ㄷ
...,...,...,...,...,...,...
548,문화 관점,정치/경제,단답형,복수정당제,미국의 정당은 양당제의 양상을 띄고있는 반면 한국의 정당은 원칙적으로 어떤 양상으로...,복수정당제#복수 정당제
549,문화 관점,과학기술,단답형,정보통신 표준화,대만의 TAICS처럼 정보통신 표준화 기준을 수립하는 한국의 기관은 어디인가요?,tta#TTA#한국정보통신기술협회#한국 정보통신 기술협회#한국 정보 통신 기술협회#...
557,문화 지식,사회,단답형,그린벨트,2026년 상반기 그린벨트가 풀리는 비수도권 전략사업 지역 중 U-밸리 일반산단이 ...,울주군#울주#울산 울주군#울산광역시 울주군
571,문화 지식,지리,단답형,대중교통,2024년 하반기부터 시작된 한국의 대중교통비 환급 지원 사업의 이름은?,K-패스#K 패스#K패스#k-패스#k 패스#k패스#케이패스#케이 패스


In [12]:
def select_short_answer(text):
    # '#'이 있는 경우, 첫 번째 항목을 선택
    if '#' in text:
        """
        '#'로 분리된 텍스트 조각 중 가장 길이가 긴 문자열을 반환합니다.
        """
        segments = text.split('#')
        # key=len 으로 각 문자열 길이를 기준으로 최대값을 구함
        longest = max(segments, key=len)
        return longest.strip()
    return text

In [10]:
training_df.loc[tmp_df[tmp_df['answer'].str.contains('#')].index, 'answer'] = tmp_df[tmp_df['answer'].str.contains('#')]['answer'].apply(select_short_answer)
training_df[training_df['question_type'] == '단답형']

,category,domain,question_type,topic_keyword,question,answer
5,문화 지식,풍습/문화유산,단답형,금줄,한국의 금줄은 무엇을 막기 위해 문이나 길 어귀에 건너질러 매나요?,부정
12,문화 실행,일상생활,단답형,상차림,한국 상차림 시 식기류를 어떻게 배열하는지 왼쪽을 기준으로 순서대로 나열하세요. 배...,ㄴ - ㄷ - ㄹ - ㄱ
13,문화 지식,역사,단답형,조선후기 실학자(정약용),조선 후기의 실학 사상가로 목민심서를 쓴 인물은?,정약용
14,문화 지식,과학기술,단답형,이태규,한국에 양자역학을 화학 반응 연구에 도입한 과학자로 이론 화학을 연구했던 과학자는 ...,이태규
16,문화 지식,역사,단답형,고조선,한국사에서 처음으로 등장한 국가는 무엇인가요?,고조선
...,...,...,...,...,...,...
557,문화 지식,사회,단답형,그린벨트,2026년 상반기 그린벨트가 풀리는 비수도권 전략사업 지역 중 U-밸리 일반산단이 ...,울산광역시 울주군
571,문화 지식,지리,단답형,대중교통,2024년 하반기부터 시작된 한국의 대중교통비 환급 지원 사업의 이름은?,케이 패스
577,문화 지식,가치관,단답형,남아선호사상,"한국의 남아선호사상은 어떤 종교나 사상을 바탕으로 확산, 강화되었나요?",유교
584,문화 관점,가치관,단답형,이사 미신,러시아에서는 이사한 집에 고양이가 드나들면 복이 온다고 합니다. 한국에서는 복을 위...,밥솥


In [13]:
print(training_df.loc[0,'question']), print(training_df.loc[0,'answer'])

다음 한국의 전통 놀이 중 '조선시대'에 행한 놀이는? 
 1.주사위 놀이 2.검무 3.격구 4.영고 5.무애무
3


(None, None)

In [14]:
training_df.to_csv('/workspace/korean_culture_QA_2025/data/preprocessed/original_train.csv', index=False)
dev_df.to_csv('/workspace/korean_culture_QA_2025/data/preprocessed/original_dev.csv', index=False)

### 4지 선다인지 5지 선다인지 모른다.

In [13]:
# import re
# import pandas as pd

# def replace_answer_in_row(
#     df: pd.DataFrame,
#     row_idx: int,
#     new_ans: int
# ) -> pd.DataFrame:
#     """
#     df: 'question'과 'answer' 컬럼을 가진 DataFrame
#     row_idx: 변경할 행의 인덱스 (정수 또는 라벨)
#     new_ans: 새 정답 번호
#     new_ans: 새 정답 번호
#     반환값: question과 answer가 갱신된 새로운 DataFrame
#     """
#     # 내부에서 사용할 선택지 교환 함수
#     def swap_options(q: str) -> str:
#         # 질문 문장과 선택지 분리
#         m = re.search(r'(\d+\.)', q)
#         if not m:
#             return q
#         idx = m.start()
#         stem = q[:idx].strip()
#         choices_str = q[idx:].strip()

#         # (번호, 텍스트) 파싱
#         pattern = r'(\d+)\.([^0-9]+?)(?=\s*\d+\.|$)'
#         matches = re.findall(pattern, choices_str)
#         options = [text.strip() for _, text in matches]

#         # 인덱스 유효성 검사
#         if not (1 <= old_ans <= len(options) and 1 <= new_ans <= len(options)):
#             raise ValueError("old_ans 또는 new_ans가 선택지 범위 밖입니다.")

#         # 두 옵션 위치 교환
#         options[old_ans-1], options[new_ans-1] = options[new_ans-1], options[old_ans-1]

#         # 재조합
#         new_choices = '\n'.join(f"{i+1}.{opt}" for i, opt in enumerate(options))
#         return f"{stem}\n{new_choices}"

#     df = df.copy()  # 원본 보존

#     # 해당 행의 question만 교체
#     original_q = df.at[row_idx, 'question']
#     old_ans = df.at[row_idx, 'answer']
#     old_ans = int(old_ans)  # 정답 번호를 정수로 변환
#     df.at[row_idx, 'question'] = swap_options(original_q)
#     df.at[row_idx, 'answer'] = new_ans

#     return df



### 랜덤하게 배치

In [14]:
# import re
# import random
# import pandas as pd

# def randomize_answer_in_row(df: pd.DataFrame, row_idx) -> pd.DataFrame:
#     """
#     df: 'question'과 'answer' 컬럼을 가진 DataFrame
#     row_idx: 변경할 행의 인덱스
#     반환: 그 행만 question/answer가 갱신된 새 DataFrame
#     """
#     # 1) 질문문과 선택지 분리
#     q = df.at[row_idx, 'question']
#     if '\n' in q:
#         stem, choices_str = q.split('\n', 1)
#     else:
#         m = re.search(r'\s+[1-5]\.', q)
#         if not m:
#             raise ValueError(f"선다형 포맷이 아닙니다:\n{q}")
#         stem, choices_str = q[:m.start()], q[m.start():]
#     stem = stem.strip()

#     # 2) 옵션 파싱: 1~5번만 인식
#     opt_pattern = re.compile(r'(?<=\s|^)([1-5])\.(.*?)(?=(?:\s+[1-5]\.)|$)', re.S)
#     parsed = opt_pattern.findall(choices_str)
#     opts = [text.strip() for _, text in parsed]
    
#     num_opts = len(opts)
#     if num_opts not in (4, 5):
#         raise ValueError(f"4지 또는 5지 선다가 아닙니다. 옵션 개수: {num_opts}\n{q}")

#     # 3) 새 정답 랜덤 선택 (기존 정답 제외)
#     orig_ans = int(df.at[row_idx, 'answer'])
#     candidates = [i for i in range(1, num_opts+1) if i != orig_ans]
#     new_ans = random.choice(candidates)

#     # 4) 옵션 위치 교환
#     opts[orig_ans-1], opts[new_ans-1] = opts[new_ans-1], opts[orig_ans-1]

#     # 5) question과 answer 갱신
#     new_choices = ' '.join(f"{i+1}.{opt}" for i, opt in enumerate(opts))
#     new_q = f"{stem}\n{new_choices}"

#     df2 = df.copy()
#     df2.at[row_idx, 'question'] = new_q
#     df2.at[row_idx, 'answer'] = new_ans
#     return df2


In [15]:
# mask = training_df['question_type'] == '선다형'
# for idx, row in training_df[mask].iterrows():
#     # 랜덤으로 정답 위치 변경
#     new_row = randomize_answer_in_row(training_df, idx)
#     training_df.loc[idx] = new_row.loc[idx]
# training_df

---
# grpo datasets

In [ ]:
import pandas as pd

In [17]:
training_df = pd.read_json("data/train.json", encoding="utf-8-sig")
dev_df = pd.read_json("data/dev.json", encoding="utf-8-sig")
test_df = pd.read_json("data/test.json", encoding="utf-8-sig")

len(training_df), len(dev_df), len(test_df)

(491, 98, 394)

In [18]:
training_df = pd.concat([training_df, dev_df], ignore_index=True)
training_df

,id,input,output
0,1,"{'category': '문화 지식', 'domain': '예술', 'questio...",{'answer': '3'}
1,2,"{'category': '문화 지식', 'domain': '풍습/문화유산', 'qu...",{'answer': '4'}
2,3,"{'category': '문화 지식', 'domain': '사회', 'questio...",{'answer': '대한민국의 행정구역은 여러 종류의 지역 단위로 나뉘어 구성되어...
3,4,"{'category': '문화 지식', 'domain': '과학기술', 'quest...",{'answer': '2'}
4,5,"{'category': '문화 지식', 'domain': '일상생활', 'quest...",{'answer': '4'}
...,...,...,...
584,585,"{'category': '문화 관점', 'domain': '가치관', 'questi...",{'answer': '밥솥'}
585,586,"{'category': '문화 지식', 'domain': '역사', 'questio...",{'answer': '2'}
586,587,"{'category': '문화 지식', 'domain': '역사', 'questio...",{'answer': '4'}
587,588,"{'category': '문화 실행', 'domain': '지리', 'questio...",{'answer': 'ㄷ-ㄴ-ㄹ-ㄱ#ㄷ - ㄴ - ㄹ - ㄱ'}


In [19]:
training_df = pd.concat([pd.DataFrame(training_df['input'].to_list()), pd.DataFrame(training_df['output'].to_list())], axis=1)
training_df

,category,domain,question_type,topic_keyword,question,answer
0,문화 지식,예술,선다형,전통놀이,다음 한국의 전통 놀이 중 '조선시대'에 행한 놀이는? \n 1\t주사위 놀이 2\...,3
1,문화 지식,풍습/문화유산,선다형,진도아리랑,한국의 진도아리랑은 어떤 타령에 영향을 받아 만들어졌나요? \n 1\t 각설이타령 ...,4
2,문화 지식,사회,서술형,행정구역,대한민국의 행정구역 체계를 서술하세요.,"대한민국의 행정구역은 여러 종류의 지역 단위로 나뉘어 구성되어 있으며, 먼저 특별시..."
3,문화 지식,과학기술,선다형,우주인,힌국인 최초의 우주 비행사는 누구인가요? \n 1\t 조니 김 2\t 이소연...,2
4,문화 지식,일상생활,선다형,돌잔치,돌잔치와 관련 없는 것을 고르세요. \n 1\t 돌잡이 2\t 돌상 3\t 돌...,4
...,...,...,...,...,...,...
584,문화 관점,가치관,단답형,이사 미신,러시아에서는 이사한 집에 고양이가 드나들면 복이 온다고 합니다. 한국에서는 복을 위...,밥솥
585,문화 지식,역사,선다형,병인양요,흥선군의 천주교 탄압에 대한 보복으로 프랑스군이 강화도에 침입한 사건은? \n 1\...,2
586,문화 지식,역사,선다형,고려시대사,다음 중 고려시대의 사건이 아닌 것은 무엇인가요? \n 1\t이자겸의 난 2\t저고...,4
587,문화 실행,지리,단답형,기후동행카드,기후동행카드를 실물로 사용하기 위한 방법을 순서에 맞게 배열하세요. 배열 시 각 기...,ㄷ-ㄴ-ㄹ-ㄱ#ㄷ - ㄴ - ㄹ - ㄱ


### 1. \t -> '.'

In [20]:
import re

def convert_number_tab_to_dot(text):
    # 정규식: 숫자 + 탭을 숫자 + 점으로 변경
    return re.sub(r'(\d+)\\t', r'\1.', text)


training_df['question'] = training_df['question'].apply(convert_number_tab_to_dot)
training_df['question'] = training_df['question'].str.replace('\\n', '\n')

training_df

,category,domain,question_type,topic_keyword,question,answer
0,문화 지식,예술,선다형,전통놀이,다음 한국의 전통 놀이 중 '조선시대'에 행한 놀이는? \n 1.주사위 놀이 2.검...,3
1,문화 지식,풍습/문화유산,선다형,진도아리랑,한국의 진도아리랑은 어떤 타령에 영향을 받아 만들어졌나요? \n 1. 각설이타령 ...,4
2,문화 지식,사회,서술형,행정구역,대한민국의 행정구역 체계를 서술하세요.,"대한민국의 행정구역은 여러 종류의 지역 단위로 나뉘어 구성되어 있으며, 먼저 특별시..."
3,문화 지식,과학기술,선다형,우주인,힌국인 최초의 우주 비행사는 누구인가요? \n 1. 조니 김 2. 이소연 ...,2
4,문화 지식,일상생활,선다형,돌잔치,돌잔치와 관련 없는 것을 고르세요. \n 1. 돌잡이 2. 돌상 3. 돌옷 ...,4
...,...,...,...,...,...,...
584,문화 관점,가치관,단답형,이사 미신,러시아에서는 이사한 집에 고양이가 드나들면 복이 온다고 합니다. 한국에서는 복을 위...,밥솥
585,문화 지식,역사,선다형,병인양요,흥선군의 천주교 탄압에 대한 보복으로 프랑스군이 강화도에 침입한 사건은? \n 1....,2
586,문화 지식,역사,선다형,고려시대사,다음 중 고려시대의 사건이 아닌 것은 무엇인가요? \n 1.이자겸의 난 2.저고여 ...,4
587,문화 실행,지리,단답형,기후동행카드,기후동행카드를 실물로 사용하기 위한 방법을 순서에 맞게 배열하세요. 배열 시 각 기...,ㄷ-ㄴ-ㄹ-ㄱ#ㄷ - ㄴ - ㄹ - ㄱ


In [21]:
training_df['answer'].value_counts()

answer
4                                                                                                                                                                                                                                                                                                                             88
2                                                                                                                                                                                                                                                                                                                             77
1                                                                                                                                                                                                                                                                                                                             77
3                             

In [22]:
training_df[training_df['question_type'] == '단답형']

,category,domain,question_type,topic_keyword,question,answer
5,문화 지식,풍습/문화유산,단답형,금줄,한국의 금줄은 무엇을 막기 위해 문이나 길 어귀에 건너질러 매나요?,부정
12,문화 실행,일상생활,단답형,상차림,한국 상차림 시 식기류를 어떻게 배열하는지 왼쪽을 기준으로 순서대로 나열하세요. 배...,ㄴ-ㄷ-ㄹ-ㄱ#ㄴ - ㄷ - ㄹ - ㄱ
13,문화 지식,역사,단답형,조선후기 실학자(정약용),조선 후기의 실학 사상가로 목민심서를 쓴 인물은?,정약용
14,문화 지식,과학기술,단답형,이태규,한국에 양자역학을 화학 반응 연구에 도입한 과학자로 이론 화학을 연구했던 과학자는 ...,이태규
16,문화 지식,역사,단답형,고조선,한국사에서 처음으로 등장한 국가는 무엇인가요?,고조선
...,...,...,...,...,...,...
557,문화 지식,사회,단답형,그린벨트,2026년 상반기 그린벨트가 풀리는 비수도권 전략사업 지역 중 U-밸리 일반산단이 ...,울주군#울주#울산 울주군#울산광역시 울주군
571,문화 지식,지리,단답형,대중교통,2024년 하반기부터 시작된 한국의 대중교통비 환급 지원 사업의 이름은?,K-패스#K 패스#K패스#k-패스#k 패스#k패스#케이패스#케이 패스
577,문화 지식,가치관,단답형,남아선호사상,"한국의 남아선호사상은 어떤 종교나 사상을 바탕으로 확산, 강화되었나요?",유교
584,문화 관점,가치관,단답형,이사 미신,러시아에서는 이사한 집에 고양이가 드나들면 복이 온다고 합니다. 한국에서는 복을 위...,밥솥


In [23]:
tmp_df = training_df[training_df['question_type'] == '단답형']
tmp_df[tmp_df['answer'].str.contains('#')]

,category,domain,question_type,topic_keyword,question,answer
12,문화 실행,일상생활,단답형,상차림,한국 상차림 시 식기류를 어떻게 배열하는지 왼쪽을 기준으로 순서대로 나열하세요. 배...,ㄴ-ㄷ-ㄹ-ㄱ#ㄴ - ㄷ - ㄹ - ㄱ
25,문화 지식,교육,단답형,초등학교,한국의 보편적인 취학 시작 연령은? 만 나이로 이야기하세요.,만 6세#만6세#만6살#만 6살#6세#6살#여섯살
34,문화 지식,예술,단답형,타령,"한국의 타령 중 판소리의 장단은 주로 2가지인데, 하나는 중중모리 장단이고 다른 하...",자진모리장단#자진모리 장단
42,문화 실행,풍습/문화유산,단답형,식혜,식혜 만드는 방법을 순서에 맞게 배열하세요. 배열 시 각 기호는 '-'로 연결합니다...,ㅁ-ㄱ-ㄷ-ㄴ-ㄹ#ㅁ - ㄱ - ㄷ - ㄴ - ㄹ
44,문화 실행,지리,단답형,지하철 노선,서울 지하철 신림선을 서원역에서 샛강 방향으로 탔을 때 지나가는 역을 순서대로 나열...,ㄱ-ㄹ-ㄴ-ㄷ#ㄱ - ㄹ - ㄴ - ㄷ
...,...,...,...,...,...,...
548,문화 관점,정치/경제,단답형,복수정당제,미국의 정당은 양당제의 양상을 띄고있는 반면 한국의 정당은 원칙적으로 어떤 양상으로...,복수정당제#복수 정당제
549,문화 관점,과학기술,단답형,정보통신 표준화,대만의 TAICS처럼 정보통신 표준화 기준을 수립하는 한국의 기관은 어디인가요?,tta#TTA#한국정보통신기술협회#한국 정보통신 기술협회#한국 정보 통신 기술협회#...
557,문화 지식,사회,단답형,그린벨트,2026년 상반기 그린벨트가 풀리는 비수도권 전략사업 지역 중 U-밸리 일반산단이 ...,울주군#울주#울산 울주군#울산광역시 울주군
571,문화 지식,지리,단답형,대중교통,2024년 하반기부터 시작된 한국의 대중교통비 환급 지원 사업의 이름은?,K-패스#K 패스#K패스#k-패스#k 패스#k패스#케이패스#케이 패스


In [24]:
# def select_short_answer(text):
#     # '#'이 있는 경우, 첫 번째 항목을 선택
#     if '#' in text:
#         """
#         '#'로 분리된 텍스트 조각 중 가장 길이가 긴 문자열을 반환합니다.
#         """
#         segments = text.split('#')
#         # key=len 으로 각 문자열 길이를 기준으로 최대값을 구함
#         longest = max(segments, key=len)
#         return longest.strip()
#     return text

In [25]:
# training_df.loc[tmp_df[tmp_df['answer'].str.contains('#')].index, 'answer'] = tmp_df[tmp_df['answer'].str.contains('#')]['answer'].apply(select_short_answer)
# training_df[training_df['question_type'] == '단답형']

In [26]:
print(training_df.loc[0,'question']), print(training_df.loc[0,'answer'])

다음 한국의 전통 놀이 중 '조선시대'에 행한 놀이는? 
 1.주사위 놀이 2.검무 3.격구 4.영고 5.무애무
3


(None, None)

In [28]:
training_df[training_df['question_type'] == '단답형']

,category,domain,question_type,topic_keyword,question,answer
5,문화 지식,풍습/문화유산,단답형,금줄,한국의 금줄은 무엇을 막기 위해 문이나 길 어귀에 건너질러 매나요?,부정
12,문화 실행,일상생활,단답형,상차림,한국 상차림 시 식기류를 어떻게 배열하는지 왼쪽을 기준으로 순서대로 나열하세요. 배...,ㄴ-ㄷ-ㄹ-ㄱ#ㄴ - ㄷ - ㄹ - ㄱ
13,문화 지식,역사,단답형,조선후기 실학자(정약용),조선 후기의 실학 사상가로 목민심서를 쓴 인물은?,정약용
14,문화 지식,과학기술,단답형,이태규,한국에 양자역학을 화학 반응 연구에 도입한 과학자로 이론 화학을 연구했던 과학자는 ...,이태규
16,문화 지식,역사,단답형,고조선,한국사에서 처음으로 등장한 국가는 무엇인가요?,고조선
...,...,...,...,...,...,...
557,문화 지식,사회,단답형,그린벨트,2026년 상반기 그린벨트가 풀리는 비수도권 전략사업 지역 중 U-밸리 일반산단이 ...,울주군#울주#울산 울주군#울산광역시 울주군
571,문화 지식,지리,단답형,대중교통,2024년 하반기부터 시작된 한국의 대중교통비 환급 지원 사업의 이름은?,K-패스#K 패스#K패스#k-패스#k 패스#k패스#케이패스#케이 패스
577,문화 지식,가치관,단답형,남아선호사상,"한국의 남아선호사상은 어떤 종교나 사상을 바탕으로 확산, 강화되었나요?",유교
584,문화 관점,가치관,단답형,이사 미신,러시아에서는 이사한 집에 고양이가 드나들면 복이 온다고 합니다. 한국에서는 복을 위...,밥솥


In [29]:
training_df.to_csv('/workspace/korean_culture_QA_2025/data/preprocessed/grpo_train.csv', index=False, encoding='utf-8-sig')

---

In [1]:
import json
import re

# 전처리 함수 정의
def convert_number_tab_to_dot(text):
    # 숫자 + 탭(\t) → 숫자 + 점(.) 으로 변환
    return re.sub(r'(\d+)\\t', r'\1.', text)

# JSON 파일 열기
with open("/workspace/korean_culture_QA_2025/data/test.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 전처리 수행
for item in data:
    question = item["input"]["question"]
    question = convert_number_tab_to_dot(question)
    question = question.replace("\\n", "\n")  # 문자열 내 '\\n'을 실제 개행 문자로 변환
    item["input"]["question"] = question

# 전처리된 JSON 저장
with open("/workspace/korean_culture_QA_2025/data/preprocessed/test.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [1]:
import json
import re

# 전처리 함수 정의
def convert_number_tab_to_dot(text):
    return re.sub(r'(\d+)\\t', r'\1.', text)

# JSON 파일 열기
with open("/workspace/korean_culture_QA_2025/data/test_with_answers.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# "question" 필드 전처리
for key in data["question"]:
    text = data["question"][key]
    text = convert_number_tab_to_dot(text)
    text = text.replace("\\n", "\n")  # '\\n'을 실제 개행문자로 변환
    data["question"][key] = text

# 전처리된 결과 저장
with open("/workspace/korean_culture_QA_2025/data/preprocessed/test_with_answers.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


---

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/workspace/korean_culture_QA_2025/data/preprocessed/grpo_train.csv')
df

,category,domain,question_type,topic_keyword,question,answer
0,문화 지식,예술,선다형,전통놀이,다음 한국의 전통 놀이 중 '조선시대'에 행한 놀이는? \n 1.주사위 놀이 2.검...,3
1,문화 지식,풍습/문화유산,선다형,진도아리랑,한국의 진도아리랑은 어떤 타령에 영향을 받아 만들어졌나요? \n 1. 각설이타령 ...,4
2,문화 지식,사회,서술형,행정구역,대한민국의 행정구역 체계를 서술하세요.,"대한민국의 행정구역은 여러 종류의 지역 단위로 나뉘어 구성되어 있으며, 먼저 특별시..."
3,문화 지식,과학기술,선다형,우주인,힌국인 최초의 우주 비행사는 누구인가요? \n 1. 조니 김 2. 이소연 ...,2
4,문화 지식,일상생활,선다형,돌잔치,돌잔치와 관련 없는 것을 고르세요. \n 1. 돌잡이 2. 돌상 3. 돌옷 ...,4
...,...,...,...,...,...,...
584,문화 관점,가치관,단답형,이사 미신,러시아에서는 이사한 집에 고양이가 드나들면 복이 온다고 합니다. 한국에서는 복을 위...,밥솥
585,문화 지식,역사,선다형,병인양요,흥선군의 천주교 탄압에 대한 보복으로 프랑스군이 강화도에 침입한 사건은? \n 1....,2
586,문화 지식,역사,선다형,고려시대사,다음 중 고려시대의 사건이 아닌 것은 무엇인가요? \n 1.이자겸의 난 2.저고여 ...,4
587,문화 실행,지리,단답형,기후동행카드,기후동행카드를 실물로 사용하기 위한 방법을 순서에 맞게 배열하세요. 배열 시 각 기...,ㄷ-ㄴ-ㄹ-ㄱ#ㄷ - ㄴ - ㄹ - ㄱ


In [3]:
df['question_type'].value_counts()

question_type
선다형    307
단답형    149
서술형    133
Name: count, dtype: int64

In [4]:
df1 = df[df['question_type'] == '선다형']
df2 = df[df['question_type'] == '단답형']
df3 = df[df['question_type'] == '서술형']

len(df1), len(df2), len(df3)

(307, 149, 133)

In [5]:
df1.to_csv('data/preprocessed/grpo_train_선다형.csv', index=False)
df2.to_csv('data/preprocessed/grpo_train_단답형.csv', index=False)
df3.to_csv('data/preprocessed/grpo_train_서술형.csv', index=False)

---

In [1]:
import pandas as pd

In [5]:
df = pd.DataFrame(pd.read_json('/workspace/korean_culture_QA_2025/data/preprocessed/test.json')['input'].to_list())
df

,category,domain,question_type,topic_keyword,question
0,문화 지식,정치/경제,단답형,사전투표,대한민국 국민이 유권자로서 선거일 이전 기간에 별도 신고 없이 어느 투표소에서나 미...
1,문화 관점,정치/경제,서술형,의무투표제,호주는 선거 투표율이 90%를 넘습니다. 이와 관련해 호주와 우리나라의 선거제도에는...
2,문화 지식,예술,선다형,한국 미술관,한국의 미술관이 아닌 곳은 어디인가요? \n1. 오월미술관 2. 이한메미술관 3. ...
3,문화 지식,정치/경제,단답형,7·4남북공동성명,남북간 정치적 대화통로를 마련하고 한반도에 평화를 정착시키기 위해 1972년에 발표...
4,문화 실행,일상생활,선다형,조리도구,일반적으로 한국에서 김치찌개를 만드는 과정이 아닌 것은? \n 1. 배추김치를 한 ...
...,...,...,...,...,...
389,문화 관점,가치관,선다형,선물하면 안 되는 것,러시아에서는 노란꽃을 연인에게 선물하면 연인이 떠나거나 바람을 핀다고 생각합니다. ...
390,문화 지식,풍습/문화유산,서술형,동지,한국의 동짓날에는 왜 팥죽을 먹나요?
391,문화 지식,지리,선다형,한국의 도시,땅끝 마을로 알려져 있으며 한반도의 땅끝임을 알리는 탑이 있는 도시 이름은 무엇인가...
392,문화 실행,일상생활,선다형,수능,한국의 대학수학능력시험에서 수험자 유의 사항이 아닌 것은? \n 1.신분증 지참 2...


In [9]:
df['id']=pd.read_json('/workspace/korean_culture_QA_2025/data/preprocessed/test.json')['id']
df

,category,domain,question_type,topic_keyword,question,id
0,문화 지식,정치/경제,단답형,사전투표,대한민국 국민이 유권자로서 선거일 이전 기간에 별도 신고 없이 어느 투표소에서나 미...,590
1,문화 관점,정치/경제,서술형,의무투표제,호주는 선거 투표율이 90%를 넘습니다. 이와 관련해 호주와 우리나라의 선거제도에는...,591
2,문화 지식,예술,선다형,한국 미술관,한국의 미술관이 아닌 곳은 어디인가요? \n1. 오월미술관 2. 이한메미술관 3. ...,592
3,문화 지식,정치/경제,단답형,7·4남북공동성명,남북간 정치적 대화통로를 마련하고 한반도에 평화를 정착시키기 위해 1972년에 발표...,593
4,문화 실행,일상생활,선다형,조리도구,일반적으로 한국에서 김치찌개를 만드는 과정이 아닌 것은? \n 1. 배추김치를 한 ...,594
...,...,...,...,...,...,...
389,문화 관점,가치관,선다형,선물하면 안 되는 것,러시아에서는 노란꽃을 연인에게 선물하면 연인이 떠나거나 바람을 핀다고 생각합니다. ...,979
390,문화 지식,풍습/문화유산,서술형,동지,한국의 동짓날에는 왜 팥죽을 먹나요?,980
391,문화 지식,지리,선다형,한국의 도시,땅끝 마을로 알려져 있으며 한반도의 땅끝임을 알리는 탑이 있는 도시 이름은 무엇인가...,981
392,문화 실행,일상생활,선다형,수능,한국의 대학수학능력시험에서 수험자 유의 사항이 아닌 것은? \n 1.신분증 지참 2...,982


In [11]:
df2 = pd.read_json('/workspace/korean_culture_QA_2025/data/preprocessed/test_with_answers_2.json')
df2

,category,domain,question_type,topic_keyword,question,only_answer
0,문화 지식,정치/경제,단답형,사전투표,대한민국 국민이 유권자로서 선거일 이전 기간에 별도 신고 없이 어느 투표소에서나 미...,사전투표제#사전투표제도#사전투표#사전투표제#사전투표
1,문화 관점,정치/경제,서술형,의무투표제,호주는 선거 투표율이 90%를 넘습니다. 이와 관련해 호주와 우리나라의 선거제도에는...,"호주는 의무투표제를 시행하여 모든 유권자가 투표에 참여해야 하고, 불참 시 벌금을 ..."
2,문화 지식,예술,선다형,한국 미술관,한국의 미술관이 아닌 곳은 어디인가요? \n1. 오월미술관 2. 이한메미술관 3. ...,4
3,문화 지식,정치/경제,단답형,7·4남북공동성명,남북간 정치적 대화통로를 마련하고 한반도에 평화를 정착시키기 위해 1972년에 발표...,7·4 남북공동성명#7·4 남북 공동 성명#7·4남북공동성명
4,문화 실행,일상생활,선다형,조리도구,일반적으로 한국에서 김치찌개를 만드는 과정이 아닌 것은? \n 1. 배추김치를 한 ...,2
...,...,...,...,...,...,...
389,문화 관점,가치관,선다형,선물하면 안 되는 것,러시아에서는 노란꽃을 연인에게 선물하면 연인이 떠나거나 바람을 핀다고 생각합니다. ...,3
390,문화 지식,풍습/문화유산,서술형,동지,한국의 동짓날에는 왜 팥죽을 먹나요?,"한국의 동짓날에 팥죽을 먹는 이유는 역사적, 문화적 배경에 있습니다. 음력으로 한 ..."
391,문화 지식,지리,선다형,한국의 도시,땅끝 마을로 알려져 있으며 한반도의 땅끝임을 알리는 탑이 있는 도시 이름은 무엇인가...,1
392,문화 실행,일상생활,선다형,수능,한국의 대학수학능력시험에서 수험자 유의 사항이 아닌 것은? \n 1.신분증 지참 2...,3


In [12]:
df2['id'] = df['id']
df2

,category,domain,question_type,topic_keyword,question,only_answer,id
0,문화 지식,정치/경제,단답형,사전투표,대한민국 국민이 유권자로서 선거일 이전 기간에 별도 신고 없이 어느 투표소에서나 미...,사전투표제#사전투표제도#사전투표#사전투표제#사전투표,590
1,문화 관점,정치/경제,서술형,의무투표제,호주는 선거 투표율이 90%를 넘습니다. 이와 관련해 호주와 우리나라의 선거제도에는...,"호주는 의무투표제를 시행하여 모든 유권자가 투표에 참여해야 하고, 불참 시 벌금을 ...",591
2,문화 지식,예술,선다형,한국 미술관,한국의 미술관이 아닌 곳은 어디인가요? \n1. 오월미술관 2. 이한메미술관 3. ...,4,592
3,문화 지식,정치/경제,단답형,7·4남북공동성명,남북간 정치적 대화통로를 마련하고 한반도에 평화를 정착시키기 위해 1972년에 발표...,7·4 남북공동성명#7·4 남북 공동 성명#7·4남북공동성명,593
4,문화 실행,일상생활,선다형,조리도구,일반적으로 한국에서 김치찌개를 만드는 과정이 아닌 것은? \n 1. 배추김치를 한 ...,2,594
...,...,...,...,...,...,...,...
389,문화 관점,가치관,선다형,선물하면 안 되는 것,러시아에서는 노란꽃을 연인에게 선물하면 연인이 떠나거나 바람을 핀다고 생각합니다. ...,3,979
390,문화 지식,풍습/문화유산,서술형,동지,한국의 동짓날에는 왜 팥죽을 먹나요?,"한국의 동짓날에 팥죽을 먹는 이유는 역사적, 문화적 배경에 있습니다. 음력으로 한 ...",980
391,문화 지식,지리,선다형,한국의 도시,땅끝 마을로 알려져 있으며 한반도의 땅끝임을 알리는 탑이 있는 도시 이름은 무엇인가...,1,981
392,문화 실행,일상생활,선다형,수능,한국의 대학수학능력시험에서 수험자 유의 사항이 아닌 것은? \n 1.신분증 지참 2...,3,982


In [ ]:
df2.to_json( '/workspace/korean_culture_QA_2025/data/preprocessed/test_with_answers_2_id.json',index=False,indent=4, force_ascii=False)

In [ ]:
import pandas as pd

test_df = pd.read_json('data/test.json')
test_df = pd.DataFrame(test_df['input'].to_list())

In [ ]:
import re
from collections import Counter


final_answers = []


for question_type, answers in zip(test_df['question_type'], all_answers_per_chunk):
    if not answers[0]:  # 정답이 없는 경우
        final_answers.append(None)
        continue

    # 첫 글자가 숫자인 답변이 있는지 확인
    if question_type == '선다형':
        # 선다형으로 처리: 숫자 부분만 비교
        # 숫자만 추출해서 가장 많이 나온 숫자 선택
        first_numbers = [re.match(r'\d', answer.strip()).group() for answer in answers if re.match(r'\d', answer.strip())]
        most_common = Counter(first_numbers).most_common(1)
        final_answer = most_common[0][0] if most_common else None
    else:
        # 단답형으로 처리: 중복 제거 후 '#' 기준으로 이어붙이기
        unique_answers = list(dict.fromkeys(answer.strip() for answer in answers if answer.strip()))
        final_answer = '#'.join(unique_answers) if unique_answers else None

    final_answers.append(final_answer)
final_answers

['사전투표제',
 None,
 '2',
 '7·4 남북 공동 성명#7.4 남북 공동 성명',
 '2',
 '2',
 '호몰로지를 이용했습니다.#호모토피 이론#대수적 위상#호몰로지 이론',
 None,
 '1',
 '3',
 '오미자',
 '1',
 '경부고속도로',
 None,
 None,
 '2',
 None,
 '2',
 None,
 '3',
 '2',
 '1',
 None,
 '까치(까치소리)#까치#부엉이',
 '2',
 None,
 '1',
 '1',
 '새만금 사업#새만금 간척 사업#새만금 간척사업',
 '상견례',
 None,
 '2',
 '3',
 '집현전#규장각',
 None,
 None,
 '성균관',
 '3',
 '1',
 '4',
 '3',
 '4',
 '4',
 None,
 None,
 '4',
 '2',
 None,
 '1',
 '천자총통#비격진천뢰',
 '찜질방#한증막',
 '순종',
 '시김새',
 '3',
 '3',
 None,
 '4',
 '현금영수증 제도',
 '1',
 '원룸',
 '2',
 '3.15 부정선거#3·15 부정선거 사건',
 None,
 None,
 '정선아리랑, 진도아리랑, 밀양아리랑#정선 아리랑, 밀양 아리랑, 진도 아리랑#정선 아리랑, 진도 아리랑, 밀양 아리랑',
 None,
 None,
 '3',
 '2',
 None,
 '양양 암흑물질 탐색 실험(COSINE-100)#코요테(CAPP) 실험#코닥(KIMS) 실험#코사인(CoGeNT) 실험 (Cosine Experiment)#코사인(CoSiNE) 실험',
 '2',
 '1',
 '1',
 'ㄹ-ㄱ-ㄷ-ㄴ',
 '3',
 '3',
 '수제천',
 '국립중앙극장#국립극장',
 None,
 None,
 '2',
 None,
 '2',
 None,
 "직지심체요절#'직지심체요절' (약칭: 직지)",
 None,
 '1',
 '4',
 '4',
 '1',
 '초등학교 6년, 중학교 3년, 고등학교 3년입니다.#초등학교 6년, 중학교 3년, 고등학교 3년#초등학교

In [ ]:
test_df['only_answer'] = final_answers
test_df

,category,domain,question_type,topic_keyword,question,only_answer
0,문화 지식,정치/경제,단답형,사전투표,대한민국 국민이 유권자로서 선거일 이전 기간에 별도 신고 없이 어느 투표소에서나 미...,사전투표제
1,문화 관점,정치/경제,서술형,의무투표제,호주는 선거 투표율이 90%를 넘습니다. 이와 관련해 호주와 우리나라의 선거제도에는...,None
2,문화 지식,예술,선다형,한국 미술관,한국의 미술관이 아닌 곳은 어디인가요? \n1\t 오월미술관 2\t 이한메미술관 3...,2
3,문화 지식,정치/경제,단답형,7·4남북공동성명,남북간 정치적 대화통로를 마련하고 한반도에 평화를 정착시키기 위해 1972년에 발표...,7·4 남북 공동 성명#7.4 남북 공동 성명
4,문화 실행,일상생활,선다형,조리도구,일반적으로 한국에서 김치찌개를 만드는 과정이 아닌 것은? \n 1\t 배추김치를 한...,2
...,...,...,...,...,...,...
389,문화 관점,가치관,선다형,선물하면 안 되는 것,러시아에서는 노란꽃을 연인에게 선물하면 연인이 떠나거나 바람을 핀다고 생각합니다. ...,3
390,문화 지식,풍습/문화유산,서술형,동지,한국의 동짓날에는 왜 팥죽을 먹나요?,None
391,문화 지식,지리,선다형,한국의 도시,땅끝 마을로 알려져 있으며 한반도의 땅끝임을 알리는 탑이 있는 도시 이름은 무엇인가...,1
392,문화 실행,일상생활,선다형,수능,한국의 대학수학능력시험에서 수험자 유의 사항이 아닌 것은? \n 1\t신분증 지참 ...,3


In [ ]:
len(test_df[test_df['question_type'] == '선다형'])

193

In [ ]:
test_df[test_df['question_type'] == '선다형']['only_answer'].apply(lambda x : x.isdigit()).value_counts()

only_answer
True    193
Name: count, dtype: int64

In [ ]:
test_df[test_df['question_type'] == '단답형']['only_answer'].apply(lambda x : x.isdigit()).value_counts()

only_answer
False    101
True       1
Name: count, dtype: int64

In [ ]:
test_df[test_df['question_type'] == '단답형'][test_df[test_df['question_type'] == '단답형']['only_answer'].apply(lambda x : x.isdigit()) == True]

,category,domain,question_type,topic_keyword,question,only_answer
213,문화 지식,일상생활,단답형,전화번호,한국의 국제전화 국가 번호는 무엇인가요?,82


In [ ]:
test_df.reset_index(inplace=True)
test_df

,index,category,domain,question_type,topic_keyword,question,only_answer
0,0,문화 지식,정치/경제,단답형,사전투표,대한민국 국민이 유권자로서 선거일 이전 기간에 별도 신고 없이 어느 투표소에서나 미...,사전투표제
1,1,문화 관점,정치/경제,서술형,의무투표제,호주는 선거 투표율이 90%를 넘습니다. 이와 관련해 호주와 우리나라의 선거제도에는...,None
2,2,문화 지식,예술,선다형,한국 미술관,한국의 미술관이 아닌 곳은 어디인가요? \n1\t 오월미술관 2\t 이한메미술관 3...,2
3,3,문화 지식,정치/경제,단답형,7·4남북공동성명,남북간 정치적 대화통로를 마련하고 한반도에 평화를 정착시키기 위해 1972년에 발표...,7·4 남북 공동 성명#7.4 남북 공동 성명
4,4,문화 실행,일상생활,선다형,조리도구,일반적으로 한국에서 김치찌개를 만드는 과정이 아닌 것은? \n 1\t 배추김치를 한...,2
...,...,...,...,...,...,...,...
389,389,문화 관점,가치관,선다형,선물하면 안 되는 것,러시아에서는 노란꽃을 연인에게 선물하면 연인이 떠나거나 바람을 핀다고 생각합니다. ...,3
390,390,문화 지식,풍습/문화유산,서술형,동지,한국의 동짓날에는 왜 팥죽을 먹나요?,None
391,391,문화 지식,지리,선다형,한국의 도시,땅끝 마을로 알려져 있으며 한반도의 땅끝임을 알리는 탑이 있는 도시 이름은 무엇인가...,1
392,392,문화 실행,일상생활,선다형,수능,한국의 대학수학능력시험에서 수험자 유의 사항이 아닌 것은? \n 1\t신분증 지참 ...,3


In [ ]:
test_df = test_df.rename(columns={'index': 'id'})

In [ ]:
test_df.to_csv('data/preprocessed/test_with_answers_no_websearch.csv', index=False)

---

In [2]:
import re

match_format = re.compile(
    r"정답:\s*(\S.*|(?:\r?\n\s*(\S.*)))"
)


In [5]:
tests = [
    # "정답: 사과",
    # "정답:   사과",
    # "정답:    \n포도\n사과",
    # "정답:\n  딸기\n사과"
    "정답:"
]

for t in tests:
    m = match_format.search(t)
    print(t, "=>", m.group(1) if m else None)


정답: => None
